In [1]:
import numpy as np
import networkx as nx
import dgl
from dgl.nn import GraphConv
import torch
import torch.nn as nn

Using backend: pytorch


In [2]:
n = 6
nxg = nx.Graph()
nxg.add_nodes_from(range(n))
E = [(0, 1), (0, 2), (1, 2), (2, 3), (2, 4), (4, 5)]
for (u, v) in E:
    nxg.add_edge(u, v)
    
g = dgl.from_networkx(nxg)
g = dgl.add_self_loop(g)
print(g)

Graph(num_nodes=6, num_edges=18,
      ndata_schemes={}
      edata_schemes={})


# (fixed) 特徴と重み

In [3]:
X = torch.tensor([[0, 0, 0], [0, 0, 1], [0, 1, 0], 
                  [0, 1, 1], [1, 0, 0], [1, 1, 0]], dtype=torch.float32)
output_dim = 2

# GCNLayerの作成

In [4]:
gcn = GraphConv(X.shape[1], output_dim, bias=False)
print(gcn)
print(gcn.weight)
print(gcn(g, X))

GraphConv(in=3, out=2, normalization=both, activation=None)
Parameter containing:
tensor([[-0.4256,  0.6477],
        [ 1.0687,  0.9229],
        [ 0.1252, -0.8146]], requires_grad=True)
tensor([[ 0.3177, -0.0333],
        [ 0.3177, -0.0333],
        [ 0.5137,  0.1757],
        [ 0.9349,  0.3460],
        [ 0.3966,  1.0954],
        [ 0.1478,  1.0497]], grad_fn=<MulBackward0>)


In [5]:
org_weight = gcn.state_dict()["weight"]
print(list(gcn.parameters()))

## another example
# Wparam = torch.nn.parameter.Parameter(
#     torch.tensor([[0., 1.], [2., 3.], [4., 5.]], dtype=torch.float32, requires_grad=True)
# )

Wparam = torch.nn.parameter.Parameter(torch.ones_like(org_weight))
gcn.weight = Wparam
print(gcn.weight)

[Parameter containing:
tensor([[-0.4256,  0.6477],
        [ 1.0687,  0.9229],
        [ 0.1252, -0.8146]], requires_grad=True)]
Parameter containing:
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)


In [6]:
print(X)
print(gcn.weight)
print(gcn(g, X))

tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 1.],
        [1., 0., 0.],
        [1., 1., 0.]])
Parameter containing:
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[0.5915, 0.5915],
        [0.5915, 0.5915],
        [1.3489, 1.3489],
        [1.3162, 1.3162],
        [1.4080, 1.4080],
        [1.4082, 1.4082]], grad_fn=<MulBackward0>)


In [7]:
Dhat = np.diag(np.power(np.sum(Ahat, axis=1), -1/2))
print(Dhat)

NameError: name 'Ahat' is not defined

In [ ]:
mat = Dhat @ Ahat @ Dhat @ X @ W
print(mat)

In [8]:
gcn0 = GraphConv(X.shape[1], output_dim, bias=False, norm="none")
gcn1 = GraphConv(X.shape[1], output_dim, bias=False, norm="right")
gcn2 = GraphConv(X.shape[1], output_dim, bias=False, norm="both")
gcn0.weight = Wparam
gcn1.weight = Wparam
gcn2.weight = Wparam

In [9]:
print(gcn0(g, X))
print(gcn1(g, X))
print(gcn2(g, X))

tensor([[2., 2.],
        [2., 2.],
        [5., 5.],
        [3., 3.],
        [4., 4.],
        [3., 3.]], grad_fn=<GSpMMBackward>)
tensor([[0.6667, 0.6667],
        [0.6667, 0.6667],
        [1.0000, 1.0000],
        [1.5000, 1.5000],
        [1.3333, 1.3333],
        [1.5000, 1.5000]], grad_fn=<MulBackward0>)
tensor([[0.5915, 0.5915],
        [0.5915, 0.5915],
        [1.3489, 1.3489],
        [1.3162, 1.3162],
        [1.4080, 1.4080],
        [1.4082, 1.4082]], grad_fn=<MulBackward0>)
